In [ ]:
import torch            
from torch import nn
import torch.nn.functional as F
import pickle
import numpy as np

In [ ]:
def load_pretrained_word2vec(emb_size):
    with open('word2id','rb') as f:
        word2id = pickle.load(f)  #2889个
    f.close()
    weights = np.zeros([len(word2id),emb_size], dtype=np.float32)
    
    word2vec = {}
    lines_num = 0
    with open('financial_bigram-char',errors='ignore') as f:
        first_line = True
        for line in f:
            if first_line:
                first_line = False
                dim = int(line.rstrip().split()[1])
                continue
            lines_num += 1
            tokens = line.rstrip().split(' ')
            word2vec[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])

    for word_i in word2id:
        if word_i in word2vec:
            weights[word2id[word_i]] = word2vec[word_i]
        else:
            weights[word2id[word_i]] = np.random.uniform(-0.1,0.1)
    weights = torch.from_numpy(weights)
    return weights
#load_pretrained_word2vec(300)
    

In [ ]:

def init_gru(gru):
    gru.reset_parameters()
    for _, hh, _, _ in gru.all_weights:
        for i in range(0, hh.size(0), gru.hidden_size):
            nn.init.orthogonal_(hh[i:i + gru.hidden_size], gain=1)



#实现model的三个环节
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, n_layers, weight):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.n_layers = n_layers
        #self.dropout = dropout
        self.embedding = nn.Embedding.from_pretrained(weight, freeze=False)
        self.gru = nn.GRU(embed_size, hidden_size, n_layers, bidirectional=True)
        init_gru(self.gru)

    def forward(self, input_seqs, input_lens, hidden=None):
        """
        forward procedure. No need for inputs to be sorted
        :param input_seqs: Variable of [T,B]
        :param hidden:
        :param input_lens: *numpy array* of len for each input sequence
        :return:
        """
        batch_size = input_seqs.size(1)
        embedded = self.embedding(input_seqs)
        embedded = embedded.transpose(0, 1)  # [B,T,E]
        
        sort_idx = np.argsort(-input_lens)
        #DELETE = cuda_(torch
        unsort_idx = (torch.LongTensor(np.argsort(sort_idx))).cuda()
        input_lens = input_lens[sort_idx]
        #DELETE = cuda_(torch
        sort_idx = (torch.LongTensor(sort_idx)).cuda()
        embedded = embedded[sort_idx].transpose(0, 1)  # [T,B,E]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lens)
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        #print(outputs.size())
        outputs = outputs.transpose(0, 1)[unsort_idx].transpose(0, 1).contiguous()
        hidden = hidden.transpose(0, 1)[unsort_idx].transpose(0, 1).contiguous()
        return outputs, hidden, embedded

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
class Attn(nn.Module):
    def __init__(self,hidden_size,d_a):
        super(Attn, self).__init__()
        self.hidden_size = hidden_size
        self.d_a = d_a
        self.W2_weights = nn.Parameter(torch.Tensor(1, self.d_a))
        torch.nn.init.xavier_uniform(self.W2_weights.data)

        self.W1 = (nn.Linear(self.hidden_size*2,self.d_a))
        
    def forward(self,encoder_outputs,output_batch):
        output = encoder_outputs
        #encoder_outputs--[length,batch,hidden*2]      
        W1 = torch.tanh(self.W1(output)) #[length,batch,da]
        W1 = W1.permute(1,0,2)#[batch,length,da]
        score = torch.bmm(W1,
                            self.W2_weights  # (1, da)
                            .permute(1, 0)  # (da, 1)
                            .unsqueeze(0)  # (1, da, 1)
                            .repeat(output_batch, 1, 1)
                            # (batch_size, da, 1)
                            )
        score = F.softmax(F.relu(score.squeeze()))
        weighted = torch.mul(output.permute(1,0,2), score.unsqueeze(-1).expand_as(output.permute(1,0,2)))
        representations = weighted.sum(1).squeeze()
        return representations

        

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
class Dynamic_route(nn.Module):
    def __init__(self,n_iter,hidden_size):
        super(Dynamic_route,self).__init__()
        self.n_iter = n_iter
        self.embedding = 2*hidden_size
        self.trans_W = nn.Linear(self.embedding,self.embedding)
        
    def forward(self, x):
        x = self.trans_W(x)#[way,shot,vector_embedding]
        logits = torch.zeros(x.size(0),x.size(1),1).cuda()                                                
        for i in range(self.n_iter):
            probs = F.softmax(logits,dim=1)
            #print(probs.size())
            probs_mul_x = torch.mul(x, probs.expand_as(x))
            y = self.squash(probs_mul_x.sum(dim=1,keepdim=True))##[way,1,vector_embedding]
            if i!= self.n_iter-1:
                delta_logits = (y * x).sum(dim=-1, keepdim=True)
                logits = logits + delta_logits
        return y  
        
    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)
        

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
class Relation_score(nn.Module):
    def __init__(self,k, batch, in_channel, out_channel):
        super(Relation_score,self).__init__()
        self.neural_tensor_net = nn.Bilinear(in_channel,out_channel,k)
        self.mlp = (nn.Linear(k,1))
    def forward(self,query,classes):
        relation_vector = self.neural_tensor_net(query,classes)
        final_score = F.sigmoid(self.mlp(relation_vector))
        #print(relation_vector.size())       
        return final_score


In [ ]:
def dict_2_2array(x):
    sq_idx = x
    
    sq_ways_text = None #把所有classes的text集合成一个二维数组
    for i in sq_idx:
        if sq_ways_text==None:
            sq_ways_text = sq_idx[i]['text_content'] 
        else:
            sq_ways_text.extend(sq_idx[i]['text_content'])
            
    sq_ways_text_lens = None
    for i in sq_idx:
        if sq_ways_text_lens==None:
            sq_ways_text_lens = sq_idx[i]['text_len'] 
        else:
            sq_ways_text_lens.extend(sq_idx[i]['text_len'])
    #print(len(sq_ways_text))

    sq_ways_text = torch.tensor(sq_ways_text)
    sq_ways_text_lens = torch.tensor(sq_ways_text_lens)
    #print(sq_ways_text.size())
    return sq_ways_text,sq_ways_text_lens

In [ ]:
def sample_sq(intent2text,intent_texts_len,way,shot,query):
    
    train_classes = list(intent2text.keys())
    sample_classes = []
    sample_classes = np.random.permutation(train_classes)[:way]
    #print(sample_classes)
    support_idx, query_idx = {},{}

    for class_i in sample_classes: 
        sample_set = []
        sample_set = intent2text[str(class_i)]#二维数组.intent对应的所有text
        text_content,text_lens ,sample_s_nums = [],[],[]
        
        #sample_s_nums找出intent2text某一个intent（sample_set）下面，数组选哪几个位置的text，位置集合
        sample_s_nums = np.random.permutation(len(sample_set))[:shot] 
        text_content = [sample_set[item] for item in sample_s_nums]
        text_lens = [intent_texts_len[str(class_i)][num] for num in sample_s_nums]
        support_idx[str(class_i)] = {'text_content':text_content,'text_len':text_lens}

        #query_set构建
        text_content,text_lens ,sample_query_nums,sample_q_nums = [],[],[],[]
        sample_query_nums = [i for i in range(len(sample_set)) if i not in sample_s_nums]#集合减小
        sample_q_nums = np.random.permutation(sample_query_nums)[:query]
        text_content = [sample_set[item] for item in sample_q_nums]
        text_lens = [intent_texts_len[str(class_i)][num] for num in sample_q_nums]
        query_idx[str(class_i)] = {'text_content':text_content,'text_len':text_lens}
    return support_idx, query_idx

In [ ]:
class Fewshot(nn.Module):
    def __init__(self, num_words, embed_size, hidden_size, emb_weight, way, shot, query, 
                 dynamic_iter,k, d_a, **kwargs):
        super(Fewshot, self).__init__()
        self.input_size = num_words
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.emb_weights = emb_weight
        self.way = way
        self.shot = shot
        self.query = query
        self.k = k
        self.encoder = Encoder(self.input_size,self.embed_size,self.hidden_size,weight=self.emb_weights,n_layers=1)
        self.att = Attn(self.hidden_size, d_a)
        self.dynamic_route = Dynamic_route(dynamic_iter,self.hidden_size)
        self.relation_score = Relation_score(k, way*query, 2*self.hidden_size, 2*self.hidden_size)
        
    def forward(self, support_text,support_text_len,query_text,query_text_len):
        output,hidden,_ = self.encoder(support_text,support_text_len)
        weighted_rep = self.att(output,output.size(1))
        C_K = weighted_rep.view(self.way, self.shot, self.hidden_size * 2)
        Classes_vectors = self.dynamic_route(C_K)
        
        output,hidden,_ = self.encoder(query_text,query_text_len)
        query_vectors = self.att(output,output.size(1))
        ex = Classes_vectors
        ex1 = (ex.repeat(1,50,1))
        loss = torch.tensor(0.0).cuda()
        for i in range(self.way):
            groud_truth = [0.0 for i in range(self.way * self.query)]
            groud_truth[self.query * i : self.query * (i+1)] = [1.0 for i in range(self.query)]
            groud_truth = torch.tensor(groud_truth).cuda()
            
            score_i = self.relation_score(query_vectors,ex1[i]).squeeze()#type,tensor
            loss += ((score_i - groud_truth)**2).sum()
        return loss
        

In [ ]:

'''
import json
import matplotlib as plt
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import random 
import jieba

with open('xinwang_train.json') as f:
    load_json = json.load(f)
    data = load_json['rasa_nlu_data']['common_examples']
for i in range(len(data)):
    tep = data[i]['intent']
    data[i]['intent'] = data[i]['text']
    data[i]['text'] = tep
#删除重复项
res_data = []
for item in data:
    if not item in res_data:
        res_data.append(item)
intentlist = []
for item in res_data:
    if not item['intent'] in intentlist:
        intentlist.append(item['intent'])
textlist = [0 for i in range(len(intentlist))]
for i in range(len(res_data)):
    if res_data[i]['intent'] in intentlist:
        textlist[intentlist.index(res_data[i]['intent'])] += 1 
        #每一个intent下面有多少个同义句
del_data = []
for item in res_data:
    if (textlist[intentlist.index(item['intent'])] >4):
        del_data.append(item)
    else:
        textlist[intentlist.index(item['intent'])]  = 0
print(len(del_data))
data = del_data
#把同义句低于5个的intent包含部分删除，intent包含的是所有的intent集合，text包含的是intent集合对应项的同义句的个数，
import pickle
intent2id = {}
for item in data:
    if not item['intent'] in intent2id:
        intent2id[item['intent']] = len(intent2id)
print(len(intent2id))
with open("intent2id",'wb') as f:
        pickle.dump(intent2id,f)
word2id={}
word2id['PAD'] = 0
word2id['UNK'] = 1
for item in data:
    seg_text = jieba.cut(item['text'],cut_all=False)
    seg_text = (list(seg_text))   
    for w in seg_text:
        if w not in word2id:
            word2id[w] = len(word2id)        
    item['text']=seg_text#使用结巴分词已经划分好
with open("word2id",'wb') as f:
        pickle.dump(word2id,f) 
 #创建word2id
data1 = data
for item in data1:
    for w in item['text']:
        if w not in word2id:
            w = 'UNK'
for i in range(2):
    print(data1[i])
with open('intent2id','rb') as f:
    intent2id = pickle.load(f)
with open('word2id','rb') as f:
    word2id = pickle.load(f)   
for item in data1:
    #print(intent2id[item['intent']])
    item['intent'] = intent2id[item['intent']]#转化成数字label
    item['text'] = [word2id[w] for w in item['text']]
random.shuffle(data1)
train_data = data1[:7300]
test_data = data1[7300:]
#可用的有len_data，便于lstm进行切分。data1。word2id()，intent2id
#embedding_weight = load_pretrained_word2vec(word2id)
#def load_pretrained_word2vec(word2id):
#做一个intent的同义句集合(intent_same记录同义句数量)，集合i位置代表intent_i的同义句有多少个，。筛选intent同义句大于15个的data条数，
intent_same = [0 for i in range(355)]
for item in train_data:
    intent_same[item['intent']] += 1
#print(intent_same)
#train_data1筛选出intent同义句>15
train_data1 = []
for item in train_data:
    if intent_same[item['intent']]>15:
        train_data1.append(item)
intent2text = {}
intent_texts_len = {}
for item in (train_data1):
    i = item['intent']
    j = item['text']
    if str(i) in intent2text:
        intent2text[str(i)].append(j)
        intent_texts_len[str(i)].append(len(j))
    else:
        intent2text[str(i)] = [j]
        intent_texts_len[str(i)] = [len(j)]
#检查text的长度分布,截断长度
text_len = [0 for i in range(100)]
for item in data1:
    if(len(item['text']))<100:
        text_len[len(item['text'])]+=1
#print(text_len)
max_len = 20
for item in data1:
    length = len(item['text'])
    if length<20:
        item['text'].extend([0] * (max_len - length))
    else:
        item['text'] = item['text'][:20]
#在每个episode，挑选5个class，每个class中选5个作为example，10个作为query，
#可用的有train_data1,intent2text(dict，intent作为key（char），text是一个二维数组),intent_texts_len
#print(intent_texts_len)
eposide_iter = 1000
way = 5
shot = 5
query = 10
train_classes = list(intent2text.keys())
#print(train_classes)
#构建sample_classes,support_idx（value是text的二维数组）,query_idx
import numpy
import torch
train_classes = list(intent2text.keys())
#for i in range(eposide_iter):
sample_classes = np.random.permutation(train_classes)[:way]
#print(sample_classes)
support_idx, query_idx = {},{}
i = 0
for class_i in sample_classes:    
    sample_set = intent2text[str(class_i)]#二维数组.intent对应的所有text
    #sample_s_nums找出intent2text某一个intent（sample_set）下面，数组选哪几个位置的text，位置集合
    sample_s_nums = np.random.permutation(len(intent2text[str(class_i)]))[:shot] 
    text_content = [sample_set[item] for item in sample_s_nums]
    text_lens = [intent_texts_len[str(class_i)][num] for num in sample_s_nums]
    support_idx[str(class_i)] = {'text_content':text_content,'text_len':text_lens}
    #support_idx[str(class_i)] = [sample_set[item] for item in sample_s_nums]
    
    #query_set构建
    sample_query_nums = [i for i in range(len(intent2text[str(class_i)])) if i not in sample_s_nums]#集合减小
    sample_q_nums = np.random.permutation(sample_query_nums)[:query]
    #query_idx[str(class_i)] = [sample_set[item] for item in sample_q_nums]
    text_content = [sample_set[item] for item in sample_q_nums]
    text_lens = [intent_texts_len[str(class_i)][num] for num in sample_q_nums]
    query_idx[str(class_i)] = {'text_content':text_content,'text_len':text_lens}
support_ways_text = None #把所有classes的text集合成一个二维数组
for i in support_idx:
    if support_ways_text==None:
        support_ways_text = support_idx[i]['text_content'] 
    else:
        support_ways_text.extend(support_idx[i]['text_content'])
support_ways_text_lens = None
for i in support_idx:
    if support_ways_text_lens==None:
        support_ways_text_lens = support_idx[i]['text_len'] 
    else:
        support_ways_text_lens.extend(support_idx[i]['text_len'])
print(len(support_ways_text))

support_ways_text = torch.tensor(support_ways_text)
support_ways_text_lens = torch.tensor(support_ways_text_lens)
print(support_ways_text.size())
#emb_weights = load_pretrained_word2vec(300)
#with open("emb_weights",'wb') as f:
 #       pickle.dump(emb_weights,f) 
with open('emb_weights','rb') as f:
    emb_weights = pickle.load(f)
import Ipynb_importer
#import Untitled1
from Untitled1 import Encoder, load_pretrained_word2vec
import pickle
with open('word2id','rb') as f:
    word2id = pickle.load(f)  #2889个词
encoder = Encoder(len(word2id),embed_size=300,hidden_size=128,n_layers=1,weight=emb_weights)
output,hidden,_ = encoder(support_ways_text.t(),support_ways_text_lens)
print(output.size(),hidden.size())
#output是(max_length, batch_size, hidden_size)（先前向后反向）
#hidden是(n_layers x num_directions, batch_size, hidden_size)
 #print(support_ways_text_lens)
output_length = output.size(0)
output_batch = output.size(1)
hidden_size = 128
d_a = 64
import torch.nn.functional as F
import torch.nn as nn
class Attn(nn.Module):
    def __init__(self,hidden_size,output_batch,d_a):
        super(Attn, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = output_batch
        self.d_a = d_a
        self.W2_weights = nn.Parameter(torch.Tensor(1, self.d_a))
        torch.nn.init.xavier_uniform(self.W2_weights.data)

        self.W1 = (nn.Linear(self.hidden_size*2,self.d_a))
        
    def forward(self,encoder_outputs):
        output = encoder_outputs
        #encoder_outputs--[length,batch,hidden*2]      
        W1 = torch.tanh(self.W1(output)) #[length,batch,da]
        W1 = W1.permute(1,0,2)#[batch,length,da]
        score = torch.bmm(W1,
                            self.W2_weights  # (1, da)
                            .permute(1, 0)  # (da, 1)
                            .unsqueeze(0)  # (1, da, 1)
                            .repeat(self.batch_size, 1, 1)
                            # (batch_size, da, 1)
                            )
        score = F.softmax(F.relu(score.squeeze()))
        weighted = torch.mul(output.permute(1,0,2), score.unsqueeze(-1).expand_as(output.permute(1,0,2)))
        representations = weighted.sum(1).squeeze()
        return representations
print(type(output),output.size())
att = Attn(hidden_size, output_batch, d_a)
print(type(att))
#from Untitled1 import Attn
att = Attn(hidden_size, output_batch, d_a)
weighted_rep = att(output)
print(weighted_rep.size())
C_K = weighted_rep.view(way,shot,hidden_size*2)
print(C_K.size())
import torch.nn.functional as F
import torch.nn as nn
class Dynamic_route(nn.Module):
    def __init__(self,n_iter,embedding):
        super(Dynamic_route,self).__init__()
        self.n_iter = n_iter
        self.embedding = embedding
        self.trans_W = nn.Linear(self.embedding,self.embedding)
        
    def forward(self, x):
        x = self.trans_W(x)#[way,shot,vector_embedding]
        logits = torch.zeros(x.size(0),x.size(1),1)#.cuda()                                                
        for i in range(self.n_iter):
            probs = F.softmax(logits,dim=1)
            print(probs.size())
            probs_mul_x = torch.mul(x, probs.expand_as(x))
            y = self.squash(probs_mul_x.sum(dim=1,keepdim=True))##[way,1,vector_embedding]
            if i!= self.n_iter-1:
                delta_logits = (y * x).sum(dim=-1, keepdim=True)
                logits = logits + delta_logits
        return y  
        
    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)
dynamic_iter = 3
#from Untitled1 import Dynamic_route
dynamic_route = Dynamic_route(dynamic_iter,C_K.size(2))
Classes_vectors = dynamic_route(C_K)
print(Classes_vectors.size())
#query_idx每一项是intent:{'text_content':[二维数组]，‘text_len’:[一维数组]}
def dict_2_2array(x):
    sq_idx = x
    sq_ways_text = None #把所有classes的text集合成一个二维数组
    for i in sq_idx:
        if sq_ways_text==None:
            sq_ways_text = sq_idx[i]['text_content'] 
        else:
            sq_ways_text.extend(sq_idx[i]['text_content'])
    sq_ways_text_lens = None
    for i in sq_idx:
        if sq_ways_text_lens==None:
            sq_ways_text_lens = sq_idx[i]['text_len'] 
        else:
            sq_ways_text_lens.extend(sq_idx[i]['text_len'])
    print(len(sq_ways_text))

    sq_ways_text = torch.tensor(sq_ways_text)
    sq_ways_text_lens = torch.tensor(sq_ways_text_lens)
    print(sq_ways_text.size())
    return sq_ways_text,sq_ways_text_lens
query_ways_text,query_ways_text_lens = dict_2_2array(query_idx)
output,hidden,_ = encoder(query_ways_text.t(),query_ways_text_lens)
#print(output.size(),hidden.size())
output_batch = output.size(1)
att = Attn(hidden_size, output_batch, d_a)
query_vectors = att(output)
print(query_vectors.size())
print(query_vectors.size())
print(Classes_vectors.size())
ex = Classes_vectors
ex1 = (ex.repeat(1,50,1))
k = 100
query_batch = query_vectors.size(0)
print(query_batch)
import torch.nn.functional as F
import torch.nn as nn
class Relation_score(nn.Module):
    def __init__(self,k, batch, in_channel, out_channel):
        super(Relation_score,self).__init__()
        self.neural_tensor_net = nn.Bilinear(in_channel,out_channel,k)
        self.mlp = (nn.Linear(k,1))
    def forward(self,query,classes):
        relation_vector = self.neural_tensor_net(query,classes)
        final_score = F.sigmoid(self.mlp(relation_vector))
        #print(relation_vector.size())       
        return final_score
print(query_vectors.size(),ex1.size())
relation_score = Relation_score(100,query_batch,query_vectors.size(1),query_vectors.size(1))
loss = torch.tensor(0.0)
for i in range(way):
    groud_truth = [0.0 for i in range(way*query)]
    groud_truth[query*i:query*(i+1)] = [1.0 for i in range(query)]
    groud_truth = torch.tensor(groud_truth)
    score_i = relation_score(query_vectors,ex1[i])#type,tensor
    
    loss += (score_i - groud_truth)**2
print(loss)
loss = torch.tensor(0.0)
for i in range(way):
    groud_truth = [0.0 for i in range(way*query)]
    groud_truth[query*i:query*(i+1)] = [1.0 for i in range(query)]
    groud_truth = torch.tensor(groud_truth)
    score_i = relation_score(query_vectors,ex1[i])#type,tensor
    print(score_i.size(),groud_truth.size(),(score_i - groud_truth)**2.size())
    loss += (score_i - groud_truth)**2
print(loss)
loss = torch.tensor(0.0)
for i in range(way):
    groud_truth = [0.0 for i in range(way*query)]
    groud_truth[query*i:query*(i+1)] = [1.0 for i in range(query)]
    groud_truth = torch.tensor(groud_truth)
    score_i = relation_score(query_vectors,ex1[i])#type,tensor
    print(score_i.size(),groud_truth.size(),((score_i - groud_truth)**2).size())
    loss += (score_i - groud_truth)**2
print(loss)
loss = torch.tensor(0.0)
for i in range(way):
    groud_truth = [0.0 for i in range(way*query)]
    groud_truth[query*i:query*(i+1)] = [1.0 for i in range(query)]
    groud_truth = torch.tensor(groud_truth)
    score_i = relation_score(query_vectors,ex1[i]).squeeze()#type,tensor
    print(score_i.size(),groud_truth.size(),((score_i - groud_truth)**2).size())
    loss += (score_i - groud_truth)**2
print(loss)
loss = torch.tensor(0.0)
for i in range(way):
    groud_truth = [0.0 for i in range(way*query)]
    groud_truth[query*i:query*(i+1)] = [1.0 for i in range(query)]
    groud_truth = torch.tensor(groud_truth)
    score_i = relation_score(query_vectors,ex1[i]).squeeze()#type,tensor
    loss += ((score_i - groud_truth)**2).sum()
print(loss)
#在每个episode，挑选5个class，每个class中选5个作为example，10个作为query，
#可用的有train_data1,intent2text(dict，intent作为key（char），text是一个二维数组),intent_texts_len
#print(intent_texts_len)
history
history
'''